In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid", {'legend.frameon':True})

columns = [
    'id', 'testing_for', 'address', 'zip_code', 'provider',
    'connected_with', 'monthly_price', 'provider_down_speed',
    'provider_price', 'actual_down_speed', 'actual_price', 'rating',
    'completed', 'created_at', 'updated_at', 'latitude', 'longitude',
    'ping', 'actual_upload_speed', 'test_id', 'ip_address', 'hostname',
    'from_mlab', 'area_code', 'test_type', 'census_code',
    'upload_median', 'download_median', 'census_status', 'test_date',
    'country_code', 'region', 'county', 'accuracy', 'location',
    'census_block'
]

# Load and filter CSV in chunks (less memory, require 5GB to load full dataset) to just from_mlab = 0 (SUA submissions)
iter_csv = pd.read_csv('./data/9035f7b8-2d2f-4de0-a816-4067e1ae8fd8.csv', header=None, names=columns, iterator=True, chunksize=1000)
sua = pd.concat([chunk[chunk['from_mlab'] == 0] for chunk in iter_csv])

# Cleanup
sua['rating'] = sua['rating'].replace(0.0,  np.nan)
sua['census_code'].dropna(inplace=True)

In [2]:
lane = sua[sua['county'] == 41039].copy()
# Convert census_codes to integer from float64
lane['census_code'] = lane['census_code'].astype(int)

In [3]:
lane.describe()

,zip_code,monthly_price,provider_down_speed,provider_price,actual_down_speed,actual_price,rating,completed,latitude,longitude,...,actual_upload_speed,from_mlab,area_code,census_code,upload_median,download_median,county,accuracy,location,census_block
count,785.000000,430.000000,368.000000,348.000000,786.000000,348.000000,500.000000,786.000000,786.000000,786.000000,...,786.000000,786.0,0.0,7.860000e+02,0.0,0.0,786.0,182.000000,0.0,7.520000e+02
mean,97423.163057,77.755814,149.312500,5.335374,58.126794,28.249511,4.164000,0.978372,44.044947,-123.115524,...,23.806489,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,1984.379121,NaN,4.103900e+14
std,30.281358,40.597156,548.745811,12.423948,77.764096,122.794332,1.887917,0.145560,0.074965,0.215943,...,75.134598,0.0,NaN,1.574336e+03,NaN,NaN,0.0,10593.162882,NaN,1.578051e+07
min,97401.000000,0.000000,0.000000,0.010000,0.030000,0.040000,1.000000,0.000000,43.686800,-124.130500,...,0.000000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,6.000000,NaN,4.103900e+14
25%,97402.000000,50.000000,12.000000,0.540000,7.722500,1.085000,3.000000,1.000000,44.034000,-123.147100,...,1.762500,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,39.250000,NaN,4.103900e+14
50%,97405.000000,70.000000,60.000000,1.330000,30.665000,2.780000,4.000000,1.000000,44.050000,-123.094100,...,5.770000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,52.000000,NaN,4.103900e+14
75%,97439.000000,93.750000,150.000000,4.260000,74.460000,10.917500,6.000000,1.000000,44.076175,-123.061250,...,11.427500,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,1062.000000,NaN,4.103900e+14
max,97490.000000,200.000000,10000.000000,120.000000,631.460000,1428.570000,7.000000,1.000000,44.262200,-122.163100,...,864.820000,0.0,NaN,4.103901e+10,NaN,NaN,41039.0,118584.000000,NaN,4.103901e+14


In [4]:
lane_tracts = lane['census_code'].unique()
len(lane_tracts)

85

## Census Tract assignments

In [5]:
# Hand built by clicking around on speedupamerica.com's results page
eugene_tracts = [
    41039002201, 41039002202,
    41039002301,
    41039002401, 41039002302, 41039002403, 41039002404,
    41039002501, 41039002503, 41039002504,
    41039002600,
    41039002700,
    41039002800,
    41039002902, 41039002903, 41039002904,
    41039003000,
    41039003101, 41039003102,
    41039003600, # Glenwood and LCC
    41039003700, # UofO
    41039003800,
    41039003900,
    41039004000,
    41039004100,
    41039004200,
    41039004300,
    41039004401, 41039004403, 41039004404, 41039004405,
    41039004501, 41039004502,
    41039004600,
    41039004700,
    41039004800,
    41039004900,
    41039005000,
    41039005100,
    41039005300,
    41039005400
]
springfield_tracts = [
    41039001801, 41039001803, 41039001804, # Thurston
    41039001902, 41039001903, 41039001904, # Centeral Springfield
    41039002001, 41039002002, # North Springfield
    41039002101, 41039002102, # Gateway
    41039003201, 41039003202, # West Springfield (Centenial)
    41039003301, 41039003302, # Springfield
    41039003400, # East of Mowhawl
    41039003500 # Jasper Rd and Dorris Ranch
]

## Eugene

In [6]:
eugene = lane[lane['census_code'].isin(eugene_tracts)]
eugene.describe()

,zip_code,monthly_price,provider_down_speed,provider_price,actual_down_speed,actual_price,rating,completed,latitude,longitude,...,actual_upload_speed,from_mlab,area_code,census_code,upload_median,download_median,county,accuracy,location,census_block
count,451.000000,214.000000,193.000000,179.000000,451.000000,179.000000,275.000000,451.000000,451.000000,451.000000,...,451.000000,451.0,0.0,4.510000e+02,0.0,0.0,451.0,122.000000,0.0,4.400000e+02
mean,97402.838137,76.177570,168.378238,3.195531,77.656430,5.250000,4.825455,0.966741,44.057102,-123.107488,...,36.512173,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,2640.434426,NaN,4.103900e+14
std,1.854235,36.544092,215.405158,10.519312,90.737394,13.049122,1.654039,0.179512,0.026685,0.032094,...,96.441290,0.0,NaN,9.013008e+02,NaN,NaN,0.0,12852.707539,NaN,8.843496e+06
min,97401.000000,0.000000,1.000000,0.010000,0.230000,0.040000,1.000000,0.000000,43.993300,-123.200500,...,0.100000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,6.000000,NaN,4.103900e+14
25%,97401.000000,50.000000,50.000000,0.340000,16.865000,0.790000,4.000000,1.000000,44.039850,-123.129700,...,4.550000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,39.250000,NaN,4.103900e+14
50%,97402.000000,70.000000,80.000000,0.830000,53.990000,1.660000,5.000000,1.000000,44.050000,-123.099000,...,8.350000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,65.000000,NaN,4.103900e+14
75%,97405.000000,90.000000,200.000000,1.715000,98.705000,3.970000,6.000000,1.000000,44.074750,-123.086400,...,11.895000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,1062.000000,NaN,4.103900e+14
max,97408.000000,200.000000,1000.000000,120.000000,631.460000,134.850000,7.000000,1.000000,44.131400,-123.024100,...,864.820000,0.0,NaN,4.103901e+10,NaN,NaN,41039.0,118584.000000,NaN,4.103901e+14


In [7]:
eugene.groupby('provider').agg({
    "id": ["count"],
    "rating":  ["mean", "median", "count"],
    "actual_down_speed": ["mean", "median", "count"],
    "monthly_price": ["mean", "median", "count"],
    "actual_price": ["mean", "median", "count"]
}).sort_values(('id', 'count'), ascending=False).head(15)

id    rating         \
                                                   count      mean median   
provider                                                                    
Comcast Cable Communications, LLC                    284  4.546512    5.0   
XS Media                                              70  6.087719    6.0   
CenturyLink Communications, LLC                       45  3.360000    4.0   
Network for Education and Research in Oregon (N...    12  6.000000    6.0   
University of Oregon                                   6  6.000000    6.0   
Verizon Wireless                                       6  3.500000    3.5   
Emerald Broadband, LLC                                 5  5.400000    7.0   
AT&T Mobility LLC                                      4  4.000000    4.0   
ATT                                                    2       NaN    NaN   
Douglas FastNet                                        2  7.000000    7.0   
Sprint Personal Communications Systems                 2  3.000000    3.0   
Cogent Communications                                  1  7.000000    7.0   
Peak Internet, LLC                                     1       NaN    NaN   
Sprint                                                 1       NaN    NaN   
T-Mobile USA, Inc.                                     1       NaN    NaN   

                                                         actual_down_speed  \
                                                   count              mean   
provider                                                                     
Comcast Cable Communications, LLC                    172         79.011444   
XS Media                                              57        133.306571   
CenturyLink Communications, LLC                       25         13.631778   
Network for Education and Research in Oregon (N...     2         55.532500   
University of Oregon                                   3         30.760000   
Verizon Wireless                                       2         43.838333   
Emerald Broadband, LLC                                 5         29.784000   
AT&T Mobility LLC                                      2          9.022500   
ATT                                                    0         86.675000   
Douglas FastNet                                        2        145.640000   
Sprint Personal Communications Systems                 1         14.455000   
Cogent Communications                                  1        265.150000   
Peak Internet, LLC                                     0          7.630000   
Sprint                                                 0         16.220000   
T-Mobile USA, Inc.                                     0         79.620000   

                                                                   \
                                                     median count   
provider                                                            
Comcast Cable Communications, LLC                    63.780   284   
XS Media                                             99.025    70   
CenturyLink Communications, LLC                       7.720    45   
Network for Education and Research in Oregon (N...   26.120    12   
University of Oregon                                  9.160     6   
Verizon Wireless                                     26.680     6   
Emerald Broadband, LLC                               28.820     5   
AT&T Mobility LLC                                     7.205     4   
ATT                                                  86.675     2   
Douglas FastNet                                     145.640     2   
Sprint Personal Communications Systems               14.455     2   
Cogent Communications                               265.150     1   
Peak Internet, LLC                                    7.630     1   
Sprint                                               16.220     1   
T-Mobile USA, Inc.                                   79.620     1   

                   

In [8]:
eugene.groupby('census_code').agg({
    "id": ["count"],
    "rating":  ["mean", "median", "count"],
    "actual_down_speed": ["mean", "median", "count"],
    "monthly_price": ["mean", "median", "count"],
    "actual_price": ["mean", "median", "count"]
})

id    rating              actual_down_speed                 \
            count      mean median count              mean   median count   
census_code                                                                 
41039002201     2  5.000000    5.0     2          7.270000    7.270     2   
41039002202    13  4.000000    4.0     6         55.710769   32.060    13   
41039002301     6  4.000000    4.0     4         65.836667   55.950     6   
41039002302    11  5.333333    6.0     6         37.504545   36.200    11   
41039002401     3  4.666667    4.0     3         32.286667   24.330     3   
41039002403     6  4.000000    4.0     2         78.623333   49.365     6   
41039002404     7  3.800000    4.0     5         52.781429   36.050     7   
41039002501     1       NaN    NaN     0        309.340000  309.340     1   
41039002503    18  4.000000    4.0     8        168.730556   58.955    18   
41039002504     1       NaN    NaN     0         57.360000   57.360     1   
41039002600     8  4.250000    3.5     4         20.253750   11.480     8   
41039002700     6  4.750000    4.5     4         52.173333   29.275     6   
41039002800     8  6.000000    7.0     5         44.885000   25.115     8   
41039002902     6  3.666667    4.0     3        100.063333   95.070     6   
41039002903     6  3.500000    3.5     4         44.403333   42.580     6   
41039002904     5  4.000000    4.0     3         38.770000   35.950     5   
41039003000    25  6.062500    6.0    16         56.012000   30.430    25   
41039003101     9  4.142857    5.0     7         38.666667   25.640     9   
41039003102    15  3.625000    4.0     8         61.338000   44.430    15   
41039003600     2  4.500000    4.5     2          4.160000    4.160     2   
41039003700     9  5.166667    5.0     6         56.054444   11.120     9   
41039003800    12  5.250000    5.0     8         44.790000   52.855    12   
41039003900    39  5.720000    7.0    25         89.730513   67.320    39   
41039004000    30  5.640000    6.0    25         64.799000   32.270    30   
41039004100     3  1.000000    1.0     3        165.500000  165.620     3   
41039004200     7  4.250000    4.5     4         68.707143   68.100     7   
41039004300     7  4.000000    4.0     2         76.881429   27.840     7   
41039004401    13  4.846154    4.0    13         80.467692   57.160    13   
41039004403     8  4.666667    4.5     6         65.550000   67.460     8   
41039004404     4  3.333333    4.0     3        103.185000   85.525     4   
41039004405    22  5.125000    5.0     8         78.640455   45.415    22   
41039004501     1  2.000000    2.0     1          1.120000    1.120     1   
41039004502    59  5.470588    6.0    34        126.292373   81.020    59   
41039004600     5       NaN    NaN     0         56.790000   41.680     5   
41039004700    33  4.263158    5.0    19         45.700909   47.810    33   
41039004800     6  3.500000    4.0     6         34.390000   22.090     6   
41039004900     4  5.000000    5.0     3         76.147500   69.280     4   
41039005000    10  2.857143    2.0     7         89.045000   51.555    10   
41039005100     3  6.000000    6.0     2        104.380000   38.370     3   
41039005300     6  5.500000    5.5     2        166.590000  178.065     6   
41039005400    12  5.000000    5.0     6         74.201667   49.260    12   

            monthly_price              actual_price                
                     mean median count         mean  median count  
census_code                                                        
41039002201     80.000000   80.0     2     3.080000   3.080     1  
41039002202     82.500000   65.0     6     6.940000   1.910     5  
41039002301     58.333333   60.0     3     0.840000   0.840     2  
41039002302     75.833333   87.5     6     1.805000   1.740     6  
41039002401     74.000000   74.0     2          NaN     NaN     0  
41039002403    102.500000  100.0     4     0.986667   0.550     3  
41039002404   

## Springfield

In [9]:
springfield = lane[lane['census_code'].isin(springfield_tracts)]
springfield.describe()

,zip_code,monthly_price,provider_down_speed,provider_price,actual_down_speed,actual_price,rating,completed,latitude,longitude,...,actual_upload_speed,from_mlab,area_code,census_code,upload_median,download_median,county,accuracy,location,census_block
count,73.000000,46.000000,44.000000,42.000000,73.000000,42.000000,53.00000,73.000000,73.000000,73.000000,...,73.000000,73.0,0.0,7.300000e+01,0.0,0.0,73.0,22.000000,0.0,6.600000e+01
mean,97477.260274,75.304348,119.022727,3.551429,65.382055,8.264762,4.00000,0.972603,44.056262,-122.991363,...,7.984384,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,599.727273,NaN,4.103900e+14
std,0.441821,29.714097,146.374081,5.469011,58.441767,21.193785,1.82925,0.164368,0.015006,0.044957,...,7.374372,0.0,NaN,6.501787e+02,NaN,NaN,0.0,1512.805819,NaN,6.441427e+06
min,97477.000000,13.000000,0.000000,0.030000,0.440000,0.380000,1.00000,0.000000,44.011700,-123.043000,...,0.260000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,11.000000,NaN,4.103900e+14
25%,97477.000000,51.250000,18.750000,0.585000,23.900000,0.837500,3.00000,1.000000,44.046200,-123.022000,...,5.080000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,22.250000,NaN,4.103900e+14
50%,97477.000000,70.000000,60.000000,1.000000,49.860000,1.705000,4.00000,1.000000,44.058200,-123.001600,...,5.770000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,50.000000,NaN,4.103900e+14
75%,97478.000000,88.000000,150.000000,2.800000,89.370000,4.392500,5.00000,1.000000,44.068300,-122.972600,...,11.380000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,65.000000,NaN,4.103900e+14
max,97478.000000,150.000000,500.000000,26.670000,260.390000,125.000000,7.00000,1.000000,44.078400,-122.874300,...,40.510000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,5985.000000,NaN,4.103900e+14


In [10]:
springfield.groupby('provider').agg({
    "id": ["count"],
    "rating":  ["mean", "median", "count"],
    "actual_down_speed": ["mean", "median", "count"],
    "monthly_price": ["mean", "median", "count"],
    "actual_price": ["mean", "median", "count"]
}).sort_values(('id', 'count'), ascending=False).head(15)

id    rating               \
                                  count      mean median count   
provider                                                         
Comcast Cable Communications, LLC    58  4.500000    5.0    42   
CenturyLink Communications, LLC      10  2.142857    1.0     7   
AVAST Software s.r.o.                 1  2.000000    2.0     1   
SoftLayer Technologies Inc.           1  1.000000    1.0     1   
Sprint                                1  3.000000    3.0     1   
Verizon Wireless                      1       NaN    NaN     0   
ViaSat,Inc.                           1  2.000000    2.0     1   

                                  actual_down_speed                \
                                               mean  median count   
provider                                                            
Comcast Cable Communications, LLC         78.518448  67.160    58   
CenturyLink Communications, LLC            7.997000   5.855    10   
AVAST Software s.r.o.                     31.720000  31.720     1   
SoftLayer Technologies Inc.               23.900000  23.900     1   
Sprint                                    76.380000  76.380     1   
Verizon Wireless                           3.460000   3.460     1   
ViaSat,Inc.                                3.390000   3.390     1   

                                  monthly_price              actual_price  \
                                           mean median count         mean   
provider                                                                    
Comcast Cable Communications, LLC     78.882353   71.5    34     2.205313   
CenturyLink Communications, LLC       65.444444   55.0     9    30.076250   
AVAST Software s.r.o.                 60.000000   60.0     1          NaN   
SoftLayer Technologies Inc.           13.000000   13.0     1     0.540000   
Sprint                                      NaN    NaN     0          NaN   
Verizon Wireless                            NaN    NaN     0          NaN   
ViaSat,Inc.                          120.000000  120.0     1    35.400000   

                                                 
                                   median count  
provider                                         
Comcast Cable Communications, LLC   1.415    32  
CenturyLink Communications, LLC    10.970     8  
AVAST Software s.r.o.                 NaN     0  
SoftLayer Technologies Inc.         0.540     1  
Sprint                                NaN     0  
Verizon Wireless                      NaN     0  
ViaSat,Inc.                        35.400     1

In [11]:
springfield.groupby('census_code').agg({
    "id": ["count"],
    "rating":  ["mean", "median", "count"],
    "actual_down_speed": ["mean", "median", "count"],
    "monthly_price": ["mean", "median", "count"],
    "actual_price": ["mean", "median", "count"]
}).sort_values(('id', 'count'), ascending=False).head(15)

id    rating              actual_down_speed                 \
            count      mean median count              mean   median count   
census_code                                                                 
41039002002     9  3.875000    3.0     8         58.564444   67.530     9   
41039002001     8  4.000000    4.0     8         50.481250   41.115     8   
41039003302     8  3.750000    4.0     8         40.868750   44.190     8   
41039001801     7  3.600000    5.0     5         55.132857    8.380     7   
41039002101     7  6.333333    7.0     3        138.570000  156.140     7   
41039002102     6  5.333333    5.0     3         58.798333   40.090     6   
41039003202     5  2.750000    2.0     4         96.022000   76.380     5   
41039003301     4  3.500000    3.5     2         57.477500   56.055     4   
41039003500     4  3.000000    3.0     2         50.515000   50.665     4   
41039001803     3  5.000000    5.0     2         83.553333   78.810     3   
41039001804     3  2.500000    2.5     2         81.046667   10.780     3   
41039003201     3  3.000000    3.0     1         27.900000   26.370     3   
41039001902     2  6.500000    6.5     2         80.055000   80.055     2   
41039001903     2  3.500000    3.5     2         40.860000   40.860     2   
41039001904     1       NaN    NaN     0          3.460000    3.460     1   

            monthly_price              actual_price                
                     mean median count         mean  median count  
census_code                                                        
41039002002     54.333333   45.0     6     1.615000   0.730     6  
41039002001     73.166667   70.0     6     1.857500   1.830     4  
41039003302     85.571429   85.0     7    11.075000   2.290     6  
41039001801     65.000000   65.0     4    35.965000   9.130     4  
41039002101     65.000000   65.0     2     2.360000   2.360     2  
41039002102     63.600000   73.0     5     3.792000   2.210     5  
41039003202     94.333333  130.0     3     0.843333   0.540     3  
41039003301    146.000000  146.0     1          NaN     NaN     0  
41039003500    105.000000   95.0     3     5.930000   4.410     3  
41039001803     90.000000   90.0     1     7.820000   7.820     1  
41039001804     82.500000   82.5     2    28.385000  28.385     2  
41039003201     50.000000   50.0     2     1.800000   1.800     2  
41039001902     80.000000   80.0     1     2.550000   2.550     1  
41039001903     72.500000   72.5     2     2.000000   2.000     2  
41039001904           NaN    NaN     0          NaN     NaN     0

## Lane County

In [12]:
county_tracts = np.setdiff1d(lane_tracts, eugene_tracts)
county_tracts = np.setdiff1d(county_tracts, springfield_tracts)

county = lane[lane['census_code'].isin(county_tracts)]
county.describe()

,zip_code,monthly_price,provider_down_speed,provider_price,actual_down_speed,actual_price,rating,completed,latitude,longitude,...,actual_upload_speed,from_mlab,area_code,census_code,upload_median,download_median,county,accuracy,location,census_block
count,261.000000,170.000000,131.000000,127.000000,262.000000,127.000000,172.000000,262.0,262.000000,262.000000,...,262.000000,262.0,0.0,2.620000e+02,0.0,0.0,262.0,38.000000,0.0,2.460000e+02
mean,97443.153257,80.405882,131.396947,8.941339,22.487481,67.275276,3.156977,1.0,44.020871,-123.163950,...,6.343702,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,679.736842,NaN,4.103900e+14
std,28.628514,47.472009,879.375729,15.438889,33.193409,196.771799,1.807654,0.0,0.121406,0.362133,...,16.514166,0.0,NaN,6.887840e+02,NaN,NaN,0.0,1764.980540,NaN,7.010074e+06
min,97402.000000,20.000000,1.000000,0.020000,0.030000,0.350000,1.000000,1.0,43.686800,-124.130500,...,0.000000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,10.000000,NaN,4.103900e+14
25%,97424.000000,45.000000,10.000000,1.460000,2.670000,3.880000,1.000000,1.0,43.946700,-123.341500,...,0.600000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,50.000000,NaN,4.103900e+14
50%,97438.000000,65.000000,15.000000,3.800000,8.110000,10.890000,3.000000,1.0,44.038750,-123.167300,...,1.660000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,50.000000,NaN,4.103900e+14
75%,97478.000000,100.000000,42.500000,8.415000,25.500000,25.915000,4.000000,1.0,44.103200,-123.003200,...,5.895000,0.0,NaN,4.103900e+10,NaN,NaN,41039.0,138.000000,NaN,4.103900e+14
max,97490.000000,200.000000,10000.000000,90.000000,176.520000,1428.570000,7.000000,1.0,44.262200,-122.163100,...,203.650000,0.0,NaN,4.103901e+10,NaN,NaN,41039.0,9752.000000,NaN,4.103901e+14


In [13]:
county.groupby('provider').agg({
    "id": ["count"],
    "rating":  ["mean", "median", "count"],
    "actual_down_speed": ["mean", "median", "count"],
    "monthly_price": ["mean", "median", "count"],
    "actual_price": ["mean", "median", "count"]
}).sort_values(('id', 'count'), ascending=False).head(15)

id    rating               \
                                       count      mean median count   
provider                                                              
CenturyLink Communications, LLC          104  2.571429    2.0    70   
Charter Communications                    53  4.636364    5.0    33   
Verizon Wireless                          25  2.200000    2.0    15   
ViaSat,Inc.                               16  1.833333    1.5    12   
Comcast Cable Communications, LLC         15  4.666667    5.0     9   
Hughes Network Systems                    13  2.333333    3.0     9   
Emerald Broadband, LLC                     9  3.625000    3.5     8   
XS Media                                   9  4.111111    4.0     9   
Hunter Communications                      8  2.750000    3.0     4   
LS Networks                                4  5.000000    4.0     3   
AT&T Mobility LLC                          3       NaN    NaN     0   
Sprint                                     1       NaN    NaN     0   
Sprint Personal Communications Systems     1       NaN    NaN     0   
tfm associates                             1       NaN    NaN     0   

                                       actual_down_speed                 \
                                                    mean   median count   
provider                                                                  
CenturyLink Communications, LLC                 6.571923    3.710   104   
Charter Communications                         49.042075   37.110    53   
Verizon Wireless                               17.265200    8.250    25   
ViaSat,Inc.                                     7.438750    5.780    16   
Comcast Cable Communications, LLC              65.039333   63.120    15   
Hughes Network Systems                          4.800769    2.100    13   
Emerald Broadband, LLC                         24.635556    7.140     9   
XS Media                                       19.324444   17.540     9   
Hunter Communications                          46.235000   13.315     8   
LS Networks                                     6.937500    2.425     4   
AT&T Mobility LLC                              12.403333    6.710     3   
Sprint                                        115.560000  115.560     1   
Sprint Personal Communications Systems          2.920000    2.920     1   
tfm associates                                 71.400000   71.400     1   

                                       monthly_price               \
                                                mean median count   
provider                                                            
CenturyLink Communications, LLC            60.652174   54.0    69   
Charter Communications                     64.333333   64.5    30   
Verizon Wireless                          117.333333  107.5    12   
ViaSat,Inc.                               144.875000  169.0    16   
Comcast Cable Communications, LLC          71.700000   70.0    10   
Hughes Network Systems                     88.750000   92.5    12   
Emerald Broadband, LLC                    125.000000  125.0     8   
XS Media                                  118.800000  119.0     5   
Hunter Communications                      71.500000   62.5     4   
LS Networks                                41.500000   42.0     4   
AT&T Mobility LLC                                NaN    NaN     0   
Sprint                                           NaN    NaN     0   
Sprint Personal Communications Systems           NaN    NaN     0   
tfm associates                                   NaN    NaN     0   

                                       actual_price                 
                                               mean   median count  
provider                                                            
CenturyLink Communications, LLC           58.635094   16.220    53  
Charter Communications                     3.446818    1.195    22  
Verizon Wireless                       

In [14]:
county.groupby('census_code').agg({
    "id": ["count"],
    "rating":  ["mean", "median", "count"],
    "actual_down_speed": ["mean", "median", "count"],
    "monthly_price": ["mean", "median", "count"],
    "actual_price": ["mean", "median", "count"]
}).sort_values(('id', 'count'), ascending=False).head(15)

id    rating              actual_down_speed                \
            count      mean median count              mean  median count   
census_code                                                                
41039000100    26  2.777778    3.0    18         10.728846   7.305    26   
41039000902    26  3.400000    4.0    20         15.187692   4.240    26   
41039001002    22  2.250000    1.5    16         12.858636   5.395    22   
41039000903    20  3.666667    3.0    12         10.041500   2.825    20   
41039001101    17  2.500000    2.0     6         26.948824  10.130    17   
41039000200    17  2.214286    2.0    14          9.742353   7.730    17   
41039000904    13  2.777778    2.0     9         21.402308  10.060    13   
41039000403    10  4.000000    4.0     4         33.751000   7.505    10   
41039000402    10  2.666667    3.0     9          6.893000   3.490    10   
41039001700     9  2.500000    2.5     2         31.138889   8.060     9   
41039000404     9  2.500000    2.0     8         33.784444  11.140     9   
41039001102     9  3.000000    4.0     5         51.748889  24.790     9   
41039001201     9  3.333333    4.0     6         30.955556   5.410     9   
41039000800     7  3.333333    3.0     6          7.031429   3.380     7   
41039001301     6  2.333333    1.0     3         43.783333  41.525     6   

            monthly_price              actual_price                
                     mean median count         mean  median count  
census_code                                                        
41039000100     52.944444   48.5    18    12.250667  15.380    15  
41039000902    105.714286   90.0    21   161.465294  30.170    17  
41039001002    124.411765  119.0    17   198.441250  31.335     8  
41039000903     58.083333   57.5    12    79.232857  10.630     7  
41039001101     72.800000   60.0     5    18.652500  16.560     4  
41039000200     86.363636   70.0    11    21.843333  11.930     9  
41039000904     77.818182   65.0    11     7.356667   4.820     9  
41039000403     61.250000   50.0     4    15.250000  16.390     3  
41039000402     91.600000   77.5    10   198.492500  18.415     8  
41039001700     54.600000   41.0     5     6.517500   6.465     4  
41039000404     67.000000   64.5     4    41.935000  41.935     2  
41039001102     73.500000   63.5     4    32.050000  27.140     4  
41039001201     71.200000   66.0     5     6.506667   7.940     3  
41039000800     84.166667   85.0     6   219.853333  50.510     3  
41039001301     57.500000   57.5     2    11.185000  11.185     2